<a href="https://colab.research.google.com/github/joshuavetos/Tessrax/blob/main/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Tessrax Contradiction Engine v4.9 (Hardened + Traced)
-----------------------------------------------------
Secure contradiction-processing runtime:
✓ Verified receipts (secure-by-default)
✓ Sandboxed, durable quarantine with fail-safe handling
✓ Signed contradiction records (delegates chaining to ledger)
✓ Runtime tracer integration (auditable heartbeat)
✓ Modular ledger contract (ILedger-compliant)
"""

import os
import json
import time
import hashlib
import threading
import traceback
from pathlib import Path
from typing import Any, Dict, Callable, List, Optional

from nacl.signing import SigningKey
from nacl.encoding import HexEncoder
from prometheus_client import Counter

from tessrax.core.interfaces import ILedger
from tessrax.core.receipts import verify_receipt, NonceRegistry, RevocationRegistry
from tessrax.core.resource_guard import ResourceMonitor, ensure_in_sandbox
from tessrax.utils.tracer import Tracer


# ------------------------------------------------------------
# Metrics
# ------------------------------------------------------------
CONTRADICTION_EVENTS_PROCESSED = Counter(
    "tessrax_contradictions_total",
    "Number of contradiction events processed by the engine"
)


# ------------------------------------------------------------
# Exceptions
# ------------------------------------------------------------
class ContradictionEngineError(Exception):
    pass


class QuarantineViolation(Exception):
    pass


# ------------------------------------------------------------
# Contradiction Engine
# ------------------------------------------------------------
class ContradictionEngine:
    """
    Hardened Contradiction Engine with ledger, tracer, and audit integration.
    """

    def __init__(
        self,
        *,
        ledger: ILedger,
        ruleset: Optional[List[Callable[[Dict[str, Any]], Optional[Dict[str, Any]]]]] = None,
        signing_key_hex: str,
        nonce_registry: NonceRegistry,
        revocation_registry: RevocationRegistry,
        name: str = "contradiction_engine",
        verify_strict: bool = True,
        quarantine_path: str = "data/quarantine.jsonl",
        metabolize_fn: Optional[Callable[[Dict[str, Any]], Optional[Dict[str, Any]]]] = None,
    ):
        if not all([ledger, signing_key_hex, nonce_registry, revocation_registry]):
            raise ContradictionEngineError(
                "Ledger, signing_key, NonceRegistry, and RevocationRegistry are required."
            )

        self.name = name
        self.ruleset = ruleset or []
        self.ledger = ledger
        self.nonce_registry = nonce_registry
        self.revocation_registry = revocation_registry
        self.verify_strict = verify_strict
        self.metabolize_fn = metabolize_fn
        self._lock = threading.Lock()

        self.signing_key = SigningKey(signing_key_hex, encoder=HexEncoder)
        self.verify_key = self.signing_key.verify_key.encode(encoder=HexEncoder).decode()

        sandbox_root = Path("data/sandbox")
        self._quarantine_path = ensure_in_sandbox(Path(quarantine_path), sandbox_root)

        # Initialize runtime tracer (asynchronous, non-blocking)
        self.tracer = Tracer(
            ledger=self.ledger,
            private_key_hex=self.signing_key_hex,
            executor_id=self.name
        )

        self._running = True
        self._monitor = ResourceMonitor("ContradictionEngine")

    # --------------------------------------------------------
    # Core Methods
    # --------------------------------------------------------

    def _sign_payload(self, payload: Dict[str, Any]) -> Dict[str, Any]:
        """Sign a payload using the engine's key."""
        serialized = json.dumps(payload, sort_keys=True).encode()
        signature = self.signing_key.sign(serialized).signature.hex()
        payload["signature"] = signature
        payload["verify_key"] = self.verify_key
        return payload

    @Tracer.trace
    def _verify_event(self, event: Dict[str, Any]) -> bool:
        """Verifies receipt validity."""
        try:
            return verify_receipt(event.get("receipt"), strict=self.verify_strict)
        except Exception as e:
            raise ContradictionEngineError(f"Receipt verification failed: {e}")

    @Tracer.trace
    def _detect(self, event: Dict[str, Any]) -> List[Dict[str, Any]]:
        """Runs all rules against event payload to detect contradictions."""
        contradictions = []
        for rule in self.ruleset:
            try:
                result = rule(event)
                if result:
                    contradictions.append(result)
            except Exception as e:
                print(f"[WARN] Rule {rule.__name__} raised error: {e}")
        return contradictions

    @Tracer.trace
    def _quarantine(self, event: Dict[str, Any], reason: str) -> None:
        """Write quarantined event to durable forensic log."""
        try:
            record = {
                "timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
                "reason": reason,
                "event": event,
            }
            with open(self._quarantine_path, "a", encoding="utf-8") as f:
                f.write(json.dumps(record) + "\n")
        except Exception as e:
            raise QuarantineViolation(f"Failed to quarantine event: {e}")

    @Tracer.trace
    def _emit(self, contradiction: Dict[str, Any]) -> None:
        """Sign and emit contradiction; ledger maintains its own chain."""
        base = {
            "timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
            "type": "contradiction",
            "payload": contradiction,
        }
        signed = self._sign_payload(base)
        self.ledger.add_event(signed)
        if self.metabolize_fn:
            try:
                self.metabolize_fn(contradiction)
            except Exception as e:
                print(f"[WARN] Metabolism failed: {e}")
        CONTRADICTION_EVENTS_PROCESSED.inc()

    # --------------------------------------------------------
    # Batch / Loop
    # --------------------------------------------------------

    def _run_once_unlocked(self, event: Dict[str, Any]) -> None:
        """Runs the full pipeline for a single event."""
        if not self._verify_event(event):
            self._quarantine(event, "Failed receipt verification")
            return

        contradictions = self._detect(event)
        for c in contradictions:
            self._emit(c)

    def run_batch(self, events: List[Dict[str, Any]]) -> None:
        """Processes multiple events with quarantine fault tolerance."""
        with self._lock:
            for ev in events:
                try:
                    self._run_once_unlocked(ev)
                except QuarantineViolation as e:
                    print(f"[CRITICAL] Quarantine write failure: {e}")
                    self.stop()
                    break

    def run_forever(self, event_source: Callable[[], Dict[str, Any]], delay: float = 1.0):
        """Continuously polls event source and processes in real time."""
        print(f"[INFO] {self.name} running...")
        while self._running:
            try:
                event = event_source()
                if event:
                    self.run_batch([event])
                time.sleep(delay)
            except KeyboardInterrupt:
                self.stop()
            except Exception as e:
                print(f"[ERROR] Loop exception: {e}")
                self._quarantine({"error": str(e)}, "Runtime failure")

    def stop(self):
        """Stops the main loop and shuts down tracer thread."""
        print(f"[INFO] {self.name} stopping...")
        self._running = False
        self.tracer.stop()

    # --------------------------------------------------------
    # Verification / Stats
    # --------------------------------------------------------

    def verify_contradiction_chain(self) -> bool:
        """Delegates full verification to the ledger."""
        if not hasattr(self.ledger, "verify_chain"):
            raise ContradictionEngineError("Ledger missing `verify_chain()` method.")
        return self.ledger.verify_chain()

    def get_stats(self) -> Dict[str, Any]:
        """Reports engine health and metrics."""
        all_events = self.ledger.get_all_events(verify=False)
        contradictions = [e for e in all_events if e.get("type") == "contradiction"]
        scars = [e for e in all_events if e.get("type") == "scar"]
        return {
            "total_contradictions": len(contradictions),
            "total_scars": len(scars),
            "quarantine_size": os.path.getsize(self._quarantine_path)
            if self._quarantine_path.exists()
            else 0,
            "chain_valid": self.ledger.verify_chain(),
        }

In [ ]:
# Review and improve _quarantine and _emit

# _quarantine method:
# - Added more detailed logging, including the event ID being quarantined.
# - Ensured the parent directory for the quarantine file exists before writing.
# - Used json.dump for writing the record to the file, which is generally safer than f.write(json.dumps(...))
#   as it handles potential encoding issues better.
# - The exception handling is already in place to catch errors during file writing and raise QuarantineViolation.
# - The use of `ensure_in_sandbox` during initialization adds a layer of security by ensuring the path is within a designated area.

# _emit method:
# - Added more detailed logging, including the contradiction ID being emitted.
# - Wrapped the ledger interaction (`self.ledger.add_event`) in a try-except block to catch potential errors.
# - Wrapped ledger errors in the custom `LedgerInteractionError` to be caught by the `@retry` decorator.
# - Added a check to ensure `self.ledger.add_event` is callable before attempting to call it.
# - Ensured that signing errors (`ContradictionEngineError` from `_sign_payload`) are re-raised to prevent attempting to emit an unsigned payload.
# - Added logging and a check around the `metabolize_fn` execution to handle potential errors gracefully without stopping the emission process.
# - Added a check to ensure `CONTRADICTION_EVENTS_PROCESSED` is a valid Counter before attempting to increment it.
# - The `@retry` decorator is applied to handle transient `LedgerInteractionError` during emission.

print("Reviewed and refined _quarantine and _emit methods.")

Reviewed and refined _quarantine and _emit methods.


# Task
Improve and rewrite the code in cell "nfFagkR1jQPM".

## Review and refactor existing code

### Subtask:
Go through the current `ContradictionEngine` class, identify areas for improvement in terms of clarity, efficiency, and adherence to best practices. This might include refining variable names, breaking down complex methods, or improving error handling.


**Reasoning**:
Review the code in cell 'nfFagkR1jQPM' to identify areas for improvement in clarity, efficiency, and adherence to best practices as outlined in the subtask instructions.



In [ ]:
# Review the code in cell 'nfFagkR1jQPM' and identify areas for improvement.
# This is a manual review process, and no code execution is needed for this step.
# The observations and potential improvements will be compiled into a list.

# Observations and potential improvements will be documented here.
# For example:
# - Variable names like 'c' in _emit could be more descriptive (e.g., 'contradiction_data').
# - The _detect method could be broken down to make individual rule execution more explicit.
# - More specific exceptions could be raised in _verify_event.
# - Consider using a more efficient method for appending to the quarantine file if it becomes very large.
# - Add type hints to method signatures for better code clarity and maintainability.
# - Ensure consistent docstrings for all methods.

**Reasoning**:
Based on the review of the code, compile a list of specific observations and potential improvements as required by the subtask.



In [ ]:
improvements = [
    "- **Variable Naming:** Variable names like 'c' in `_emit` could be more descriptive (e.g., `contradiction_data`). The `ruleset` variable could perhaps be named `contradiction_rules` for slightly more clarity.",
    "- **Method Simplification:** The `_detect` method is reasonably concise, but if the logic for handling individual rule results were to become more complex, it might warrant breaking down. The `_run_once_unlocked` method orchestrates the single-event pipeline; while not overly complex now, adding more steps could make it a candidate for further decomposition.",
    "- **Error Handling:** While `try-except` is used, more specific exception types could be raised in methods like `_verify_event` and within individual rules in `_detect` to provide more granular error information. The `QuarantineViolation` is a good start for specific errors related to quarantine.",
    "- **Efficiency:** Appending to the quarantine file (`_quarantine`) line by line might become inefficient for very large numbers of quarantined events. Consider alternatives like batch writing or using a more suitable database if the scale increases significantly. File path operations using `Path` are good, but ensure robust error handling for file operations.",
    "- **Best Practices (PEP 8, Type Hinting, Docstrings):**\n  - Add type hints to method signatures and variable declarations for improved code readability and maintainability.\n  - Ensure all methods have clear and concise docstrings explaining their purpose, arguments, and return values.\n  - Review for full adherence to PEP 8 guidelines (line length, spacing, etc.).",
    "- **Redundant Attributes:** The `signing_key_hex` attribute is stored but can be accessed directly from `self.signing_key.encode(encoder=HexEncoder).decode()`. While minor, removing the stored attribute reduces potential for inconsistency.",
    "- **Initialization:** The `__init__` method has a check for `None` values, which is good. However, it might be beneficial to also check the *types* of the inputs where appropriate.",
    "- **Concurrency:** The use of `threading.Lock` in `run_batch` is appropriate for protecting shared state during batch processing. Ensure this lock is sufficient if more complex threading or multiprocessing is introduced later.",
    "- **Tracer Integration:** The `@Tracer.trace` decorator is used, which is good for auditing. Ensure the Tracer handles potential exceptions robustly so it doesn't disrupt the main engine flow.",
    "- **Metrics:** The Prometheus counter is well-placed in `_emit`. Consider adding more counters for different types of events (e.g., verification failures, quarantine events, rule errors)."
]

for improvement in improvements:
    print(improvement)

- **Variable Naming:** Variable names like 'c' in `_emit` could be more descriptive (e.g., `contradiction_data`). The `ruleset` variable could perhaps be named `contradiction_rules` for slightly more clarity.
- **Method Simplification:** The `_detect` method is reasonably concise, but if the logic for handling individual rule results were to become more complex, it might warrant breaking down. The `_run_once_unlocked` method orchestrates the single-event pipeline; while not overly complex now, adding more steps could make it a candidate for further decomposition.
- **Error Handling:** While `try-except` is used, more specific exception types could be raised in methods like `_verify_event` and within individual rules in `_detect` to provide more granular error information. The `QuarantineViolation` is a good start for specific errors related to quarantine.
- **Efficiency:** Appending to the quarantine file (`_quarantine`) line by line might become inefficient for very large numbers of q

## Enhance security

### Subtask:
Strengthen security aspects, potentially by reviewing the usage of `nacl`, ensuring proper handling of keys, and examining the signing and verification processes.


**Reasoning**:
Examine the code related to `nacl` for signing and verification, focusing on the `_sign_payload` and `_verify_event` methods, and the handling of the signing key.



In [ ]:
# Examine the usage of nacl in _sign_payload and _verify_event.
# The _sign_payload method serializes the payload to JSON, signs it using self.signing_key,
# and appends the signature and verify_key to the payload. This seems correct for signing.
# The _verify_event method uses verify_receipt, which is assumed to handle nacl verification internally.
# It is important that verify_receipt properly uses the verify_key included in the signed payload
# and handles potential exceptions securely (e.g., invalid signature, tampered data).
# The signing_key_hex is stored as an instance variable self.signing_key, which is a nacl.signing.SigningKey object.
# The original hex string is not stored directly after initialization, which is good.
# The verify_key is derived from the signing_key and stored as self.verify_key.

# Potential areas for strengthening:
# 1. The verify_receipt function (not shown in this cell) is critical. Its implementation needs to be
#    secure and correctly use nacl.verify.
# 2. Key management: The signing_key is passed as a hex string during initialization. In a real-world
#    scenario, this key should be loaded from a secure source (e.g., environment variable, secret management system)
#    and never hardcoded or exposed unnecessarily. The current code doesn't show the origin of signing_key_hex,
#    which is a potential security risk outside this class's scope but important to note.
# 3. Key rotation: No mechanism for key rotation is mentioned or implemented. This is a standard security practice.
# 4. Secure storage: The quarantine file path is within a sandbox, but the security of the ledger
#    (handled by ILedger) and other data storage mechanisms used by the engine is critical.

print("Reviewed nacl usage and key handling in ContradictionEngine.")
print("Identified critical dependency on secure implementation of verify_receipt.")
print("Noted potential improvements in key management (secure loading, rotation) and overall data storage security.")

Reviewed nacl usage and key handling in ContradictionEngine.
Identified critical dependency on secure implementation of verify_receipt.
Noted potential improvements in key management (secure loading, rotation) and overall data storage security.


## Improve error handling and robustness

### Subtask:
Make the engine more resilient to errors by adding more specific exception handling, logging, and potentially implementing retry mechanisms for external interactions (like with the ledger).


**Reasoning**:
Define custom exceptions for specific error scenarios and implement basic logging configuration.



In [ ]:
import logging

# Configure basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Define custom exceptions
class VerificationError(ContradictionEngineError):
    """Raised when receipt verification fails."""
    pass

class RuleExecutionError(ContradictionEngineError):
    """Raised when an error occurs during rule execution."""
    pass

class LedgerInteractionError(ContradictionEngineError):
    """Raised when an error occurs interacting with the ledger."""
    pass

NameError: name 'ContradictionEngineError' is not defined

**Reasoning**:
The previous command failed because `ContradictionEngineError` was not defined. I need to include the definition of `ContradictionEngineError` from the original code cell along with the new custom exceptions and logging configuration.



In [ ]:
import logging

# Configure basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Define base exception for the engine
class ContradictionEngineError(Exception):
    pass

# Define custom exceptions
class VerificationError(ContradictionEngineError):
    """Raised when receipt verification fails."""
    pass

class RuleExecutionError(ContradictionEngineError):
    """Raised when an error occurs during rule execution."""
    pass

class LedgerInteractionError(ContradictionEngineError):
    """Raised when an error occurs interacting with the ledger."""
    pass

**Reasoning**:
Now that the custom exceptions are defined, I need to modify the `ContradictionEngine` class methods to use these specific exceptions and add detailed logging. I will update `_verify_event`, `_detect`, `_quarantine`, and `_emit` to incorporate the new exceptions and logging. I will also add a retry mechanism to `_emit` for ledger interactions.



In [ ]:
import time
import random
from functools import wraps

# Retry decorator for transient errors
def retry(exceptions, tries=3, delay=1, backoff=2):
    """Retry calling the decorated function using an exponential backoff."""
    def deco_retry(f):
        @wraps(f)
        def f_retry(*args, **kwargs):
            mtries, mdelay = tries, delay
            while mtries > 1:
                try:
                    return f(*args, **kwargs)
                except exceptions as e:
                    logger.warning(f"Retrying {f.__name__} after {e}: {mdelay}s delay, {mtries-1} tries left")
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
            return f(*args, **kwargs) # Final attempt
        return f_retry
    return deco_retry


class ContradictionEngine:
    """
    Hardened Contradiction Engine with ledger, tracer, and audit integration.
    """

    def __init__(
        self,
        *,
        ledger: ILedger,
        ruleset: Optional[List[Callable[[Dict[str, Any]], Optional[Dict[str, Any]]]]] = None,
        signing_key_hex: str,
        nonce_registry: NonceRegistry,
        revocation_registry: RevocationRegistry,
        name: str = "contradiction_engine",
        verify_strict: bool = True,
        quarantine_path: str = "data/quarantine.jsonl",
        metabolize_fn: Optional[Callable[[Dict[str, Any]], Optional[Dict[str, Any]]]] = None,
    ):
        if not all([ledger, signing_key_hex, nonce_registry, revocation_registry]):
            logger.error("Ledger, signing_key, NonceRegistry, and RevocationRegistry are required.")
            raise ContradictionEngineError(
                "Ledger, signing_key, NonceRegistry, and RevocationRegistry are required."
            )

        self.name = name
        self.ruleset = ruleset or []
        self.ledger = ledger
        self.nonce_registry = nonce_registry
        self.revocation_registry = revocation_registry
        self.verify_strict = verify_strict
        self.metabolize_fn = metabolize_fn
        self._lock = threading.Lock()

        try:
            self.signing_key = SigningKey(signing_key_hex, encoder=HexEncoder)
            self.verify_key = self.signing_key.verify_key.encode(encoder=HexEncoder).decode()
            logger.info("Successfully initialized signing and verification keys.")
        except Exception as e:
            logger.critical(f"Failed to initialize signing keys: {e}", exc_info=True)
            raise ContradictionEngineError("Failed to initialize signing keys.") from e


        sandbox_root = Path("data/sandbox")
        try:
            self._quarantine_path = ensure_in_sandbox(Path(quarantine_path), sandbox_root)
            logger.info(f"Quarantine path set to: {self._quarantine_path}")
        except Exception as e:
             logger.critical(f"Failed to set up sandboxed quarantine path: {e}", exc_info=True)
             raise ContradictionEngineError("Failed to set up sandboxed quarantine path.") from e


        # Initialize runtime tracer (asynchronous, non-blocking)
        try:
            self.tracer = Tracer(
                ledger=self.ledger,
                private_key_hex=signing_key_hex, # Use the hex string for tracer init if needed
                executor_id=self.name
            )
            logger.info("Tracer initialized successfully.")
        except Exception as e:
             logger.error(f"Failed to initialize tracer: {e}", exc_info=True)
             # Do not raise error here, tracer is not critical path


        self._running = True
        self._monitor = ResourceMonitor("ContradictionEngine")
        logger.info(f"ContradictionEngine '{self.name}' initialized.")


    # --------------------------------------------------------
    # Core Methods
    # --------------------------------------------------------

    def _sign_payload(self, payload: Dict[str, Any]) -> Dict[str, Any]:
        """Sign a payload using the engine's key."""
        try:
            serialized = json.dumps(payload, sort_keys=True).encode()
            signature = self.signing_key.sign(serialized).signature.hex()
            payload["signature"] = signature
            payload["verify_key"] = self.verify_key
            logger.debug("Payload signed successfully.")
            return payload
        except Exception as e:
            logger.error(f"Failed to sign payload: {e}", exc_info=True)
            raise ContradictionEngineError("Failed to sign payload.") from e


    @Tracer.trace
    def _verify_event(self, event: Dict[str, Any]) -> bool:
        """Verifies receipt validity."""
        receipt = event.get("receipt")
        if not receipt:
            logger.warning("Event missing receipt.")
            return False # Or raise a specific error if missing receipt is critical

        try:
            is_valid = verify_receipt(receipt, strict=self.verify_strict)
            if not is_valid:
                logger.warning(f"Receipt verification failed for event: {event.get('id', 'N/A')}")
                # Decide if failed verification should raise an error or just return False
                # Currently returning False and quarantining in _run_once_unlocked
            else:
                 logger.debug(f"Receipt verification succeeded for event: {event.get('id', 'N/A')}")
            return is_valid
        except Exception as e: # Catch potential errors within verify_receipt
            logger.error(f"Error during receipt verification for event {event.get('id', 'N/A')}: {e}", exc_info=True)
            raise VerificationError(f"Receipt verification failed: {e}") from e


    @Tracer.trace
    def _detect(self, event: Dict[str, Any]) -> List[Dict[str, Any]]:
        """Runs all rules against event payload to detect contradictions."""
        contradictions = []
        for rule in self.ruleset:
            rule_name = rule.__name__ if hasattr(rule, '__name__') else 'anonymous_rule'
            try:
                logger.debug(f"Executing rule: {rule_name}")
                result = rule(event)
                if result:
                    logger.info(f"Rule '{rule_name}' detected contradiction.")
                    contradictions.append(result)
            except Exception as e:
                logger.error(f"Rule '{rule_name}' execution failed for event {event.get('id', 'N/A')}: {e}", exc_info=True)
                # Depending on requirements, you might want to raise RuleExecutionError here
                # or just log and continue with other rules. Current logic logs and continues.
        return contradictions


    @Tracer.trace
    def _quarantine(self, event: Dict[str, Any], reason: str) -> None:
        """Write quarantined event to durable forensic log."""
        logger.warning(f"Quarantining event {event.get('id', 'N/A')} due to: {reason}")
        try:
            record = {
                "timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
                "reason": reason,
                "event": event,
            }
            # Ensure directory exists before writing
            self._quarantine_path.parent.mkdir(parents=True, exist_ok=True)
            with open(self._quarantine_path, "a", encoding="utf-8") as f:
                json.dump(record, f) # Use json.dump for robustness
                f.write("\n")
            logger.info(f"Event quarantined successfully to {self._quarantine_path}")
        except Exception as e:
            logger.critical(f"Failed to quarantine event {event.get('id', 'N/A')}: {e}", exc_info=True)
            # Raising QuarantineViolation here is appropriate as it's a critical failure
            raise QuarantineViolation(f"Failed to quarantine event: {e}") from e

    @Tracer.trace
    @retry(exceptions=LedgerInteractionError, tries=5, delay=2, backoff=3) # Apply retry to ledger interaction
    def _emit(self, contradiction: Dict[str, Any]) -> None:
        """Sign and emit contradiction; ledger maintains its own chain."""
        logger.info("Attempting to emit contradiction to ledger.")
        base = {
            "timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
            "type": "contradiction",
            "payload": contradiction,
        }
        try:
            signed = self._sign_payload(base)
        except ContradictionEngineError as e:
            logger.error(f"Failed to sign contradiction before emitting: {e}", exc_info=True)
            # Decide if signing failure should prevent emission or be quarantined
            # For now, re-raise as it's a critical step before ledger interaction
            raise e

        try:
            self.ledger.add_event(signed)
            logger.info("Contradiction successfully added to ledger.")
        except Exception as e:
            logger.error(f"Ledger interaction failed: {e}", exc_info=True)
            # Wrap ledger errors in our custom exception for retry decorator
            raise LedgerInteractionError(f"Failed to add event to ledger: {e}") from e

        if self.metabolize_fn:
            try:
                logger.debug("Executing metabolism function.")
                self.metabolize_fn(contradiction)
                logger.debug("Metabolism function executed.")
            except Exception as e:
                logger.warning(f"Metabolism function failed for contradiction: {e}", exc_info=True)
                # Metabolism failure is logged but does not stop the process

        CONTRADICTION_EVENTS_PROCESSED.inc()
        logger.info("Contradiction processed count incremented.")

    # --------------------------------------------------------
    # Batch / Loop
    # --------------------------------------------------------

    def _run_once_unlocked(self, event: Dict[str, Any]) -> None:
        """Runs the full pipeline for a single event."""
        event_id = event.get('id', 'N/A')
        logger.info(f"Processing event: {event_id}")
        try:
            if not self._verify_event(event):
                self._quarantine(event, "Failed receipt verification")
                return
        except VerificationError as e:
             logger.error(f"Critical verification error for event {event_id}: {e}", exc_info=True)
             self._quarantine(event, f"Critical verification error: {e}")
             return
        except Exception as e:
             # Catch any other unexpected errors during verification step
             logger.critical(f"Unexpected error during verification for event {event_id}: {e}", exc_info=True)
             self._quarantine(event, f"Unexpected verification error: {e}")
             return


        try:
            contradictions = self._detect(event)
            if contradictions:
                logger.info(f"Detected {len(contradictions)} contradictions for event {event_id}.")
            else:
                 logger.info(f"No contradictions detected for event {event_id}.")
        except RuleExecutionError as e:
             logger.error(f"Rule execution error during detection for event {event_id}: {e}", exc_info=True)
             self._quarantine(event, f"Rule execution error: {e}")
             return
        except Exception as e:
             # Catch any other unexpected errors during detection step
             logger.critical(f"Unexpected error during detection for event {event_id}: {e}", exc_info=True)
             self._quarantine(event, f"Unexpected detection error: {e}")
             return


        for c in contradictions:
            try:
                self._emit(c)
            except (LedgerInteractionError, ContradictionEngineError) as e:
                logger.error(f"Failed to emit contradiction for event {event_id} after retries: {e}", exc_info=True)
                self._quarantine(event, f"Failed to emit contradiction: {e}")
            except Exception as e:
                 logger.critical(f"Unexpected error during emission for event {event_id}: {e}", exc_info=True)
                 self._quarantine(event, f"Unexpected emission error: {e}")


        logger.info(f"Finished processing event: {event_id}")


    def run_batch(self, events: List[Dict[str, Any]]) -> None:
        """Processes multiple events with quarantine fault tolerance."""
        logger.info(f"Processing batch of {len(events)} events.")
        with self._lock:
            for ev in events:
                try:
                    self._run_once_unlocked(ev)
                except QuarantineViolation as e:
                    # This is a critical failure - quarantine itself failed
                    logger.critical(f"Quarantine write failure: {e}. Stopping engine.", exc_info=True)
                    print(f"[CRITICAL] Quarantine write failure: {e}")
                    self.stop()
                    break
                except Exception as e:
                     # Catch any unexpected errors during batch processing loop
                     logger.critical(f"Unexpected error processing event in batch: {e}", exc_info=True)
                     # Decide if unexpected error should stop the batch or just log and continue
                     # Currently logs and continues to the next event in the batch


    def run_forever(self, event_source: Callable[[], Dict[str, Any]], delay: float = 1.0):
        """Continuously polls event source and processes in real time."""
        logger.info(f"ContradictionEngine '{self.name}' running in forever mode.")
        print(f"[INFO] {self.name} running...")
        while self._running:
            try:
                event = event_source()
                if event:
                    logger.debug("Received event from source.")
                    self.run_batch([event])
                else:
                    logger.debug("No event from source, sleeping.")
                time.sleep(delay)
            except KeyboardInterrupt:
                logger.info("Keyboard interrupt received. Stopping engine.")
                self.stop()
            except Exception as e:
                logger.error(f"Runtime loop exception: {e}", exc_info=True)
                # Decide if runtime loop exception should be quarantined or just logged
                # Currently logs and continues, but could add quarantine here
                # self._quarantine({"error": str(e)}, "Runtime failure")

        logger.info(f"ContradictionEngine '{self.name}' has stopped.")


    def stop(self):
        """Stops the main loop and shuts down tracer thread."""
        logger.info(f"ContradictionEngine '{self.name}' is stopping.")
        print(f"[INFO] {self.name} stopping...")
        self._running = False
        try:
            self.tracer.stop()
            logger.info("Tracer stopped.")
        except Exception as e:
            logger.error(f"Failed to stop tracer: {e}", exc_info=True)

    # --------------------------------------------------------
    # Verification / Stats
    # --------------------------------------------------------

    def verify_contradiction_chain(self) -> bool:
        """Delegates full verification to the ledger."""
        logger.info("Initiating ledger chain verification.")
        if not hasattr(self.ledger, "verify_chain"):
            logger.error("Ledger missing `verify_chain()` method.")
            raise ContradictionEngineError("Ledger missing `verify_chain()` method.")
        try:
            is_valid = self.ledger.verify_chain()
            if is_valid:
                logger.info("Ledger chain verification succeeded.")
            else:
                logger.warning("Ledger chain verification failed.")
            return is_valid
        except Exception as e:
            logger.error(f"Error during ledger chain verification: {e}", exc_info=True)
            raise LedgerInteractionError(f"Ledger chain verification failed: {e}") from e


    def get_stats(self) -> Dict[str, Any]:
        """Reports engine health and metrics."""
        logger.info("Gathering engine stats.")
        try:
            all_events = self.ledger.get_all_events(verify=False)
            contradictions = [e for e in all_events if e.get("type") == "contradiction"]
            scars = [e for e in all_events if e.get("type") == "scar"]

            quarantine_size = 0
            if self._quarantine_path.exists():
                try:
                    quarantine_size = os.path.getsize(self._quarantine_path)
                except Exception as e:
                    logger.warning(f"Failed to get quarantine file size: {e}")
                    quarantine_size = -1 # Indicate error getting size


            chain_valid = False
            try:
                 chain_valid = self.ledger.verify_chain()
            except Exception as e:
                 logger.error(f"Failed to verify chain for stats: {e}", exc_info=True)
                 chain_valid = False # Assume invalid if verification fails


            stats = {
                "total_contradictions": len(contradictions),
                "total_scars": len(scars),
                "quarantine_size_bytes": quarantine_size,
                "chain_valid": chain_valid,
                "engine_running": self._running,
                # Add more stats as needed
            }
            logger.info(f"Engine stats: {stats}")
            return stats
        except Exception as e:
            logger.error(f"Failed to gather engine stats: {e}", exc_info=True)
            raise ContradictionEngineError("Failed to gather engine stats.") from e


NameError: name 'ILedger' is not defined

**Reasoning**:
The previous command failed because `ILedger`, `NonceRegistry`, `RevocationRegistry`, `Tracer`, `ResourceMonitor`, `verify_receipt`, `HexEncoder`, `SigningKey`, `Counter`, `Path`, `os`, `json`, `threading`, `traceback`, `hashlib`, `Callable`, `List`, `Optional`, `Dict`, and `Any` were not defined. I need to include all necessary imports from the original cell and ensure all required classes and functions are available before defining and modifying the `ContradictionEngine` class.



In [ ]:
import os
import json
import time
import hashlib
import threading
import traceback
import random
import logging
from pathlib import Path
from typing import Any, Dict, Callable, List, Optional
from functools import wraps

from nacl.signing import SigningKey
from nacl.encoding import HexEncoder
from prometheus_client import Counter

# Assuming these are defined elsewhere and need to be imported or are mock/placeholder classes
# from tessrax.core.interfaces import ILedger # Assuming this is available or a mock is used
# from tessrax.core.receipts import verify_receipt, NonceRegistry, RevocationRegistry # Assuming these are available or mocks are used
# from tessrax.core.resource_guard import ResourceMonitor, ensure_in_sandbox # Assuming these are available or mocks are used
# from tessrax.utils.tracer import Tracer # Assuming this is available or a mock is used

# --- Mock/Placeholder classes and functions for demonstration ---
# In a real scenario, these would be imported from tessrax
class ILedger:
    def add_event(self, event):
        print(f"[MOCK_LEDGER] Adding event: {event.get('type')}")
        # Simulate a transient error occasionally
        if random.random() < 0.1:
            raise Exception("Simulated Ledger Error")
        pass

    def verify_chain(self):
         print("[MOCK_LEDGER] Verifying chain...")
         # Simulate occasional verification failure
         return random.random() > 0.05

    def get_all_events(self, verify=False):
        print("[MOCK_LEDGER] Getting all events.")
        # Return some mock events
        return [{"type": "contradiction", "id": "mock_c_1"}, {"type": "scar", "id": "mock_s_1"}]

class NonceRegistry:
    def check_and_add(self, nonce, source):
        print(f"[MOCK_NONCE_REGISTRY] Checking/adding nonce {nonce} from {source}")
        return True # Always valid for mock

class RevocationRegistry:
    def is_revoked(self, cert_id):
        print(f"[MOCK_REVOCATION_REGISTRY] Checking revocation for {cert_id}")
        return False # Never revoked for mock

class ResourceMonitor:
    def __init__(self, name):
        print(f"[MOCK_RESOURCE_MONITOR] Initialized {name}")
    def __enter__(self):
        print("[MOCK_RESOURCE_MONITOR] Starting monitor.")
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("[MOCK_RESOURCE_MONITOR] Stopping monitor.")

def ensure_in_sandbox(path, sandbox_root):
     print(f"[MOCK_SANDBOX] Ensuring {path} is in sandbox {sandbox_root}")
     # Simulate creating the sandbox directory
     sandbox_root.mkdir(parents=True, exist_ok=True)
     # Simulate returning a path within the sandbox
     return sandbox_root / path.name

class Tracer:
    def __init__(self, ledger, private_key_hex, executor_id):
        print(f"[MOCK_TRACER] Initialized for {executor_id}")
        self._running = True
        self._thread = threading.Thread(target=self._run)
        self._thread.daemon = True
        self._thread.start()

    def _run(self):
        print("[MOCK_TRACER] Running...")
        while self._running:
            time.sleep(5) # Simulate tracing work
            # print("[MOCK_TRACER] Tracing...")
        print("[MOCK_TRACER] Stopped.")

    def stop(self):
        print("[MOCK_TRACER] Stopping...")
        self._running = False
        if self._thread.is_alive():
            self._thread.join(timeout=1)

    @staticmethod
    def trace(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # print(f"[MOCK_TRACER] Tracing call to {func.__name__}")
            return func(*args, **kwargs)
        return wrapper

def verify_receipt(receipt, strict=True):
    print(f"[MOCK_VERIFY_RECEIPT] Verifying receipt (strict={strict})...")
    # Simulate occasional verification failure
    return random.random() > 0.02


# Metrics (re-initialize or use existing if already defined)
# Ensure this doesn't re-register if already run
try:
    CONTRADICTION_EVENTS_PROCESSED = Counter(
        "tessrax_contradictions_total",
        "Number of contradiction events processed by the engine"
    )
except ValueError:
    # Counter already registered, use the existing one
    from prometheus_client import REGISTRY
    CONTRADICTION_EVENTS_PROCESSED = REGISTRY.get_sample_value(
        "tessrax_contradictions_total_total" # Prometheus adds '_total' suffix
    )
    if CONTRADICTION_EVENTS_PROCESSED is None:
         # If getting the sample value fails (e.g., different registry), try getting the metric directly
         CONTRADICTION_EVENTS_PROCESSED = REGISTRY._names_to_collectors["tessrax_contradictions_total"]


# ------------------------------------------------------------
# Exceptions (Ensure base is defined first)
# ------------------------------------------------------------
class ContradictionEngineError(Exception):
    pass

class QuarantineViolation(Exception):
    pass

# Define custom exceptions
class VerificationError(ContradictionEngineError):
    """Raised when receipt verification fails."""
    pass

class RuleExecutionError(ContradictionEngineError):
    """Raised when an error occurs during rule execution."""
    pass

class LedgerInteractionError(ContradictionEngineError):
    """Raised when an error occurs interacting with the ledger."""
    pass

# ------------------------------------------------------------
# Contradiction Engine
# ------------------------------------------------------------

# Retry decorator for transient errors
def retry(exceptions, tries=3, delay=1, backoff=2):
    """Retry calling the decorated function using an exponential backoff."""
    def deco_retry(f):
        @wraps(f)
        def f_retry(*args, **kwargs):
            mtries, mdelay = tries, delay
            while mtries > 1:
                try:
                    return f(*args, **kwargs)
                except exceptions as e:
                    logger.warning(f"Retrying {f.__name__} after {e}: {mdelay:.2f}s delay, {mtries-1} tries left")
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
                except Exception as e:
                    # Log unexpected errors during retry attempts but re-raise immediately
                    logger.error(f"Unexpected error during retry attempt for {f.__name__}: {e}", exc_info=True)
                    raise
            return f(*args, **kwargs) # Final attempt
        return f_retry
    return deco_retry


class ContradictionEngine:
    """
    Hardened Contradiction Engine with ledger, tracer, and audit integration.
    """

    def __init__(
        self,
        *,
        ledger: ILedger,
        ruleset: Optional[List[Callable[[Dict[str, Any]], Optional[Dict[str, Any]]]]] = None,
        signing_key_hex: str,
        nonce_registry: NonceRegistry,
        revocation_registry: RevocationRegistry,
        name: str = "contradiction_engine",
        verify_strict: bool = True,
        quarantine_path: str = "data/quarantine.jsonl",
        metabolize_fn: Optional[Callable[[Dict[str, Any]], Optional[Dict[str, Any]]]] = None,
    ):
        logger.info(f"Initializing ContradictionEngine '{name}'.")
        if not all([ledger, signing_key_hex, nonce_registry, revocation_registry]):
            logger.error("Missing required initialization parameters.")
            raise ContradictionEngineError(
                "Ledger, signing_key, NonceRegistry, and RevocationRegistry are required."
            )

        self.name = name
        self.ruleset = ruleset or []
        self.ledger = ledger
        self.nonce_registry = nonce_registry
        self.revocation_registry = revocation_registry
        self.verify_strict = verify_strict
        self.metabolize_fn = metabolize_fn
        self._lock = threading.Lock()

        try:
            self.signing_key = SigningKey(bytes.fromhex(signing_key_hex)) # Use bytes.fromhex
            self.verify_key = self.signing_key.verify_key.encode(encoder=HexEncoder).decode()
            logger.info("Successfully initialized signing and verification keys.")
        except Exception as e:
            logger.critical(f"Failed to initialize signing keys: {e}", exc_info=True)
            raise ContradictionEngineError("Failed to initialize signing keys.") from e


        sandbox_root = Path("data/sandbox")
        try:
            # Ensure the sandbox root exists
            sandbox_root.mkdir(parents=True, exist_ok=True)
            self._quarantine_path = ensure_in_sandbox(Path(quarantine_path), sandbox_root)
            logger.info(f"Quarantine path set to: {self._quarantine_path}")
        except Exception as e:
             logger.critical(f"Failed to set up sandboxed quarantine path: {e}", exc_info=True)
             raise ContradictionEngineError("Failed to set up sandboxed quarantine path.") from e


        # Initialize runtime tracer (asynchronous, non-blocking)
        try:
            self.tracer = Tracer(
                ledger=self.ledger,
                private_key_hex=signing_key_hex,
                executor_id=self.name
            )
            logger.info("Tracer initialized successfully.")
        except Exception as e:
             logger.error(f"Failed to initialize tracer: {e}", exc_info=True)
             # Do not raise error here, tracer is not critical path


        self._running = True
        self._monitor = ResourceMonitor("ContradictionEngine")
        logger.info(f"ContradictionEngine '{self.name}' initialization complete.")


    # --------------------------------------------------------
    # Core Methods
    # --------------------------------------------------------

    def _sign_payload(self, payload: Dict[str, Any]) -> Dict[str, Any]:
        """Sign a payload using the engine's key."""
        try:
            serialized = json.dumps(payload, sort_keys=True).encode()
            # Use .sign without .signature to get the signed message object
            signed_message = self.signing_key.sign(serialized)
            payload["signature"] = signed_message.signature.hex() # Extract hex signature
            payload["verify_key"] = self.verify_key
            logger.debug("Payload signed successfully.")
            return payload
        except Exception as e:
            logger.error(f"Failed to sign payload: {e}", exc_info=True)
            raise ContradictionEngineError("Failed to sign payload.") from e


    @Tracer.trace
    def _verify_event(self, event: Dict[str, Any]) -> bool:
        """Verifies receipt validity."""
        receipt = event.get("receipt")
        event_id = event.get('id', 'N/A')
        if not receipt:
            logger.warning(f"Event {event_id} missing receipt.")
            # Depending on your policy, missing receipt might be a VerificationError
            # raise VerificationError(f"Event {event_id} missing receipt.")
            return False # Allow processing events without receipts, but mark as unverified


        try:
            is_valid = verify_receipt(receipt, strict=self.verify_strict)
            if not is_valid:
                logger.warning(f"Receipt verification failed for event: {event_id}")
            else:
                 logger.debug(f"Receipt verification succeeded for event: {event_id}")
            return is_valid
        except Exception as e: # Catch potential errors within verify_receipt
            logger.error(f"Error during receipt verification for event {event_id}: {e}", exc_info=True)
            # Raising VerificationError indicates a problem with the verification process itself
            raise VerificationError(f"Receipt verification failed for event {event_id}: {e}") from e


    @Tracer.trace
    def _detect(self, event: Dict[str, Any]) -> List[Dict[str, Any]]:
        """Runs all rules against event payload to detect contradictions."""
        contradictions = []
        event_id = event.get('id', 'N/A')
        logger.debug(f"Running contradiction detection rules for event: {event_id}")
        for rule in self.ruleset:
            rule_name = rule.__name__ if hasattr(rule, '__name__') else 'anonymous_rule'
            try:
                logger.debug(f"Executing rule '{rule_name}' for event: {event_id}")
                result = rule(event)
                if result:
                    logger.info(f"Rule '{rule_name}' detected contradiction for event {event_id}.")
                    contradictions.append(result)
            except Exception as e:
                logger.error(f"Rule '{rule_name}' execution failed for event {event_id}: {e}", exc_info=True)
                # Log the error and continue to the next rule.
                # If a single rule failure should stop processing, raise RuleExecutionError here.
        logger.debug(f"Finished running contradiction detection rules for event: {event_id}. Found {len(contradictions)}.")
        return contradictions


    @Tracer.trace
    def _quarantine(self, event: Dict[str, Any], reason: str) -> None:
        """Write quarantined event to durable forensic log."""
        event_id = event.get('id', 'N/A')
        logger.warning(f"Quarantining event {event_id} due to: {reason}")
        try:
            record = {
                "timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
                "reason": reason,
                "event": event,
            }
            # Ensure directory exists before writing
            self._quarantine_path.parent.mkdir(parents=True, exist_ok=True)
            with open(self._quarantine_path, "a", encoding="utf-8") as f:
                json.dump(record, f) # Use json.dump for robustness
                f.write("\n")
            logger.info(f"Event {event_id} quarantined successfully to {self._quarantine_path}")
        except Exception as e:
            logger.critical(f"Failed to quarantine event {event_id}: {e}", exc_info=True)
            # Raising QuarantineViolation here is appropriate as it's a critical failure
            raise QuarantineViolation(f"Failed to quarantine event {event_id}: {e}") from e

    @Tracer.trace
    @retry(exceptions=LedgerInteractionError, tries=5, delay=2, backoff=3) # Apply retry to ledger interaction
    def _emit(self, contradiction: Dict[str, Any]) -> None:
        """Sign and emit contradiction; ledger maintains its own chain."""
        contradiction_id = contradiction.get('id', 'N/A') # Use contradiction id if available
        logger.info(f"Attempting to emit contradiction {contradiction_id} to ledger.")
        base = {
            "timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
            "type": "contradiction",
            "payload": contradiction,
        }
        try:
            signed = self._sign_payload(base)
        except ContradictionEngineError as e:
            logger.error(f"Failed to sign contradiction {contradiction_id} before emitting: {e}", exc_info=True)
            # Re-raise signing failure as it's a critical step before ledger interaction
            raise e

        try:
            self.ledger.add_event(signed)
            logger.info(f"Contradiction {contradiction_id} successfully added to ledger.")
        except Exception as e:
            logger.error(f"Ledger interaction failed for contradiction {contradiction_id}: {e}", exc_info=True)
            # Wrap ledger errors in our custom exception for retry decorator
            raise LedgerInteractionError(f"Failed to add contradiction {contradiction_id} to ledger: {e}") from e

        if self.metabolize_fn:
            try:
                logger.debug(f"Executing metabolism function for contradiction {contradiction_id}.")
                self.metabolize_fn(contradiction)
                logger.debug(f"Metabolism function executed for contradiction {contradiction_id}.")
            except Exception as e:
                logger.warning(f"Metabolism function failed for contradiction {contradiction_id}: {e}", exc_info=True)
                # Metabolism failure is logged but does not stop the process

        # CONTRADICTION_EVENTS_PROCESSED.inc() # Increment counter here after successful emission
        # Check if CONTRADICTION_EVENTS_PROCESSED is a Counter metric before incrementing
        if isinstance(CONTRADICTION_EVENTS_PROCESSED, Counter):
             CONTRADICTION_EVENTS_PROCESSED.inc()
             logger.debug("Contradiction processed count incremented.")
        else:
             logger.warning("Metrics counter 'CONTRADICTION_EVENTS_PROCESSED' is not a valid Counter instance.")


    # --------------------------------------------------------
    # Batch / Loop
    # --------------------------------------------------------

    def _run_once_unlocked(self, event: Dict[str, Any]) -> None:
        """Runs the full pipeline for a single event."""
        event_id = event.get('id', 'N/A')
        logger.info(f"Processing event: {event_id}")
        try:
            # _verify_event can raise VerificationError or return False
            if not self._verify_event(event):
                # If verification returns False, quarantine the event with a specific reason
                self._quarantine(event, "Receipt verification failed (returned False)")
                logger.warning(f"Event {event_id} quarantined due to verification failure.")
                return
        except VerificationError as e:
             logger.error(f"Critical verification error for event {event_id}: {e}", exc_info=True)
             self._quarantine(event, f"Critical verification error: {e}")
             return
        except Exception as e:
             # Catch any other unexpected errors during verification step
             logger.critical(f"Unexpected error during verification for event {event_id}: {e}", exc_info=True)
             self._quarantine(event, f"Unexpected verification error: {e}")
             return


        try:
            contradictions = self._detect(event)
            if contradictions:
                logger.info(f"Detected {len(contradictions)} contradictions for event {event_id}.")
            else:
                 logger.info(f"No contradictions detected for event {event_id}.")
        except Exception as e: # Catch any error during detection (including potential RuleExecutionError from rules)
             logger.error(f"Error during contradiction detection for event {event_id}: {e}", exc_info=True)
             # Decide if detection error should quarantine the event
             self._quarantine(event, f"Error during detection: {e}")
             return


        for c in contradictions:
            try:
                self._emit(c)
            except (LedgerInteractionError, ContradictionEngineError) as e:
                # These are errors we specifically handle (ledger failure after retries, or signing failure)
                logger.error(f"Failed to emit contradiction for event {event_id} after retries or signing error: {e}", exc_info=True)
                # Quarantining the original event that led to the contradiction
                self._quarantine(event, f"Failed to emit contradiction ({c.get('id', 'N/A')}): {e}")
            except Exception as e:
                 # Catch any other unexpected errors during emission
                 logger.critical(f"Unexpected error during emission for event {event_id}: {e}", exc_info=True)
                 # Quarantining the original event
                 self._quarantine(event, f"Unexpected emission error: {e}")


        logger.info(f"Finished processing event: {event_id}")


    def run_batch(self, events: List[Dict[str, Any]]) -> None:
        """Processes multiple events with quarantine fault tolerance."""
        logger.info(f"Processing batch of {len(events)} events.")
        with self._lock:
            for i, ev in enumerate(events):
                event_id = ev.get('id', f'batch_idx_{i}')
                try:
                    logger.debug(f"Processing event {event_id} in batch.")
                    self._run_once_unlocked(ev)
                except QuarantineViolation as e:
                    # This is a critical failure - quarantine itself failed
                    logger.critical(f"Quarantine write failure processing event {event_id}: {e}. Stopping engine.", exc_info=True)
                    print(f"[CRITICAL] Quarantine write failure: {e}")
                    self.stop()
                    break # Stop processing the batch if quarantine fails
                except Exception as e:
                     # Catch any unexpected errors during the batch processing loop for a single event
                     logger.critical(f"Unexpected error processing event {event_id} in batch: {e}", exc_info=True)
                     # Log and continue to the next event in the batch


    def run_forever(self, event_source: Callable[[], Dict[str, Any]], delay: float = 1.0):
        """Continuously polls event source and processes in real time."""
        logger.info(f"ContradictionEngine '{self.name}' running in forever mode.")
        print(f"[INFO] {self.name} running...")
        while self._running:
            try:
                event = event_source()
                if event:
                    # logger.debug("Received event from source.") # Too noisy if event source is polled frequently
                    self.run_batch([event])
                else:
                    # logger.debug("No event from source, sleeping.") # Too noisy
                    pass # Just sleep if no event
                time.sleep(delay)
            except KeyboardInterrupt:
                logger.info("Keyboard interrupt received. Stopping engine.")
                self.stop()
            except Exception as e:
                logger.error(f"Runtime loop exception: {e}", exc_info=True)
                # Decide if runtime loop exception should be quarantined or just logged
                # Currently logs and continues, but could add quarantine here if needed
                # self._quarantine({"error": str(e), "traceback": traceback.format_exc()}, "Runtime loop failure")

        logger.info(f"ContradictionEngine '{self.name}' has stopped.")


    def stop(self):
        """Stops the main loop and shuts down tracer thread."""
        logger.info(f"ContradictionEngine '{self.name}' is stopping.")
        print(f"[INFO] {self.name} stopping...")
        self._running = False
        try:
            # Give the tracer a moment to finish any pending tasks
            if self.tracer and hasattr(self.tracer, 'stop'):
                 self.tracer.stop()
                 logger.info("Tracer stop requested.")
            # self.tracer.join(timeout=5) # If tracer was a joinable thread, join it
        except Exception as e:
            logger.error(f"Failed to stop tracer cleanly: {e}", exc_info=True)
        logger.info(f"ContradictionEngine '{self.name}' stop process complete.")

    # --------------------------------------------------------
    # Verification / Stats
    # --------------------------------------------------------

    def verify_contradiction_chain(self) -> bool:
        """Delegates full verification to the ledger."""
        logger.info("Initiating ledger chain verification.")
        if not hasattr(self.ledger, "verify_chain") or not callable(self.ledger.verify_chain):
            logger.error("Ledger object missing or has uncallable `verify_chain()` method.")
            raise ContradictionEngineError("Ledger missing or has uncallable `verify_chain()` method.")
        try:
            is_valid = self.ledger.verify_chain()
            if is_valid:
                logger.info("Ledger chain verification succeeded.")
            else:
                logger.warning("Ledger chain verification failed.")
            return is_valid
        except Exception as e:
            logger.error(f"Error during ledger chain verification: {e}", exc_info=True)
            raise LedgerInteractionError(f"Ledger chain verification failed: {e}") from e


    def get_stats(self) -> Dict[str, Any]:
        """Reports engine health and metrics."""
        logger.info("Gathering engine stats.")
        stats = {
            "total_contradictions": 0,
            "total_scars": 0,
            "quarantine_size_bytes": 0,
            "chain_valid": False,
            "engine_running": self._running,
            "status": "ok"
        }
        try:
            if hasattr(self.ledger, 'get_all_events') and callable(self.ledger.get_all_events):
                try:
                    all_events = self.ledger.get_all_events(verify=False)
                    stats["total_contradictions"] = len([e for e in all_events if e.get("type") == "contradiction"])
                    stats["total_scars"] = len([e for e in all_events if e.get("type") == "scar"])
                except Exception as e:
                    logger.error(f"Failed to get events from ledger for stats: {e}", exc_info=True)
                    stats["status"] = "error_ledger_stats"

            if self._quarantine_path.exists():
                try:
                    stats["quarantine_size_bytes"] = os.path.getsize(self._quarantine_path)
                except Exception as e:
                    logger.warning(f"Failed to get quarantine file size: {e}")
                    stats["quarantine_size_bytes"] = -1 # Indicate error getting size
                    stats["status"] = "warning_quarantine_size"


            try:
                 stats["chain_valid"] = self.verify_contradiction_chain()
            except LedgerInteractionError as e:
                 logger.error(f"Failed to verify chain for stats: {e}", exc_info=True)
                 stats["chain_valid"] = False # Assume invalid if verification fails
                 stats["status"] = "error_chain_verify_stats"
            except Exception as e:
                 logger.error(f"Unexpected error verifying chain for stats: {e}", exc_info=True)
                 stats["chain_valid"] = False # Assume invalid if verification fails
                 stats["status"] = "error_chain_verify_stats_unexpected"


            logger.info(f"Engine stats gathered: {stats}")
            return stats
        except Exception as e:
            logger.error(f"Unexpected error during stats gathering: {e}", exc_info=True)
            stats["status"] = "error_unexpected_stats"
            # Decide if you raise the exception or return partial stats
            # raise ContradictionEngineError("Failed to gather engine stats.") from e
            return stats # Return partial stats in case of unexpected error


ModuleNotFoundError: No module named 'nacl'

**Reasoning**:
The previous command failed because the `nacl` library is not installed in the environment. I need to install `nacl` and `prometheus_client` using `pip` before attempting to import and use them.



In [ ]:
!pip install pynacl prometheus_client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.7 MB/s eta 0:00:00


**Reasoning**:
The necessary packages have been installed. Now I can re-execute the code to define the custom exceptions, logging, mock dependencies, and the modified `ContradictionEngine` class with enhanced error handling and retries.



In [ ]:
import os
import json
import time
import hashlib
import threading
import traceback
import random
import logging
from pathlib import Path
from typing import Any, Dict, Callable, List, Optional
from functools import wraps

from nacl.signing import SigningKey
from nacl.encoding import HexEncoder
from prometheus_client import Counter, REGISTRY

# --- Mock/Placeholder classes and functions for demonstration ---
# In a real scenario, these would be imported from tessrax
class ILedger:
    def add_event(self, event):
        logger.info(f"[MOCK_LEDGER] Adding event: {event.get('type')}")
        # Simulate a transient error occasionally
        if random.random() < 0.1:
            logger.warning("[MOCK_LEDGER] Simulated transient error on add_event.")
            raise Exception("Simulated Ledger Error")
        pass

    def verify_chain(self):
         logger.info("[MOCK_LEDGER] Verifying chain...")
         # Simulate occasional verification failure
         if random.random() < 0.05:
             logger.warning("[MOCK_LEDGER] Simulated verification failure.")
             return False
         return True

    def get_all_events(self, verify=False):
        logger.info("[MOCK_LEDGER] Getting all events.")
        # Return some mock events
        return [{"type": "contradiction", "id": "mock_c_1"}, {"type": "scar", "id": "mock_s_1"}]

class NonceRegistry:
    def check_and_add(self, nonce, source):
        logger.debug(f"[MOCK_NONCE_REGISTRY] Checking/adding nonce {nonce} from {source}")
        return True # Always valid for mock

class RevocationRegistry:
    def is_revoked(self, cert_id):
        logger.debug(f"[MOCK_REVOCATION_REGISTRY] Checking revocation for {cert_id}")
        return False # Never revoked for mock

class ResourceMonitor:
    def __init__(self, name):
        logger.info(f"[MOCK_RESOURCE_MONITOR] Initialized {name}")
    def __enter__(self):
        logger.debug("[MOCK_RESOURCE_MONITOR] Starting monitor.")
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        logger.debug("[MOCK_RESOURCE_MONITOR] Stopping monitor.")

def ensure_in_sandbox(path, sandbox_root):
     logger.debug(f"[MOCK_SANDBOX] Ensuring {path} is in sandbox {sandbox_root}")
     # Simulate creating the sandbox directory
     sandbox_root.mkdir(parents=True, exist_ok=True)
     # Simulate returning a path within the sandbox
     return sandbox_root / path.name

class Tracer:
    def __init__(self, ledger, private_key_hex, executor_id):
        logger.info(f"[MOCK_TRACER] Initialized for {executor_id}")
        self._running = True
        # self._thread = threading.Thread(target=self._run)
        # self._thread.daemon = True
        # self._thread.start()
        # Using a dummy run method for simplicity in this context
        print("[MOCK_TRACER] Initialized (thread not started in mock).")


    def _run(self):
        logger.debug("[MOCK_TRACER] Running...")
        while self._running:
            time.sleep(5) # Simulate tracing work
            logger.debug("[MOCK_TRACER] Tracing...")
        logger.debug("[MOCK_TRACER] Stopped.")

    def stop(self):
        logger.info("[MOCK_TRACER] Stopping...")
        self._running = False
        # if self._thread.is_alive():
        #     self._thread.join(timeout=1)
        print("[MOCK_TRACER] Stop requested.")


    @staticmethod
    def trace(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # logger.debug(f"[MOCK_TRACER] Tracing call to {func.__name__}")
            return func(*args, **kwargs)
        return wrapper

def verify_receipt(receipt, strict=True):
    logger.debug(f"[MOCK_VERIFY_RECEIPT] Verifying receipt (strict={strict})...")
    # Simulate occasional verification failure
    if random.random() < 0.02:
        logger.warning("[MOCK_VERIFY_RECEIPT] Simulated verification failure.")
        return False
    # Simulate occasional error during verification
    if random.random() < 0.01:
        logger.error("[MOCK_VERIFY_RECEIPT] Simulated error during verification.")
        raise Exception("Simulated Verification Error")
    return True


# Metrics (re-initialize or use existing if already defined)
# Ensure this doesn't re-register if already run
try:
    CONTRADICTION_EVENTS_PROCESSED = Counter(
        "tessrax_contradictions_total",
        "Number of contradiction events processed by the engine"
    )
    logger.info("Prometheus Counter 'tessrax_contradictions_total' registered.")
except ValueError:
    # Counter already registered, try to get it from the registry
    try:
        CONTRADICTION_EVENTS_PROCESSED = REGISTRY._names_to_collectors["tessrax_contradictions_total"]
        logger.info("Prometheus Counter 'tessrax_contradictions_total' retrieved from registry.")
    except KeyError:
        logger.error("Prometheus Counter 'tessrax_contradictions_total' not found in registry after ValueError.")
        # Fallback to a dummy object if retrieving fails
        class DummyCounter:
            def inc(self, amount=1):
                logger.warning("DummyCounter 'inc' called as real counter not available.")
        CONTRADICTION_EVENTS_PROCESSED = DummyCounter()


# ------------------------------------------------------------
# Exceptions (Ensure base is defined first)
# ------------------------------------------------------------
class ContradictionEngineError(Exception):
    pass

class QuarantineViolation(Exception):
    pass

# Define custom exceptions
class VerificationError(ContradictionEngineError):
    """Raised when receipt verification fails."""
    pass

class RuleExecutionError(ContradictionEngineError):
    """Raised when an error occurs during rule execution."""
    pass

class LedgerInteractionError(ContradictionEngineError):
    """Raised when an error occurs interacting with the ledger."""
    pass

# ------------------------------------------------------------
# Contradiction Engine
# ------------------------------------------------------------

# Retry decorator for transient errors
def retry(exceptions, tries=3, delay=1, backoff=2):
    """Retry calling the decorated function using an exponential backoff."""
    def deco_retry(f):
        @wraps(f)
        def f_retry(*args, **kwargs):
            mtries, mdelay = tries, delay
            while mtries > 1:
                try:
                    return f(*args, **kwargs)
                except exceptions as e:
                    logger.warning(f"Retrying {f.__name__} after {type(e).__name__}: {e} - {mdelay:.2f}s delay, {mtries-1} tries left")
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
                except Exception as e:
                    # Log unexpected errors during retry attempts but re-raise immediately
                    logger.error(f"Unexpected error during retry attempt for {f.__name__}: {type(e).__name__} - {e}", exc_info=True)
                    raise
            # Final attempt outside the loop
            logger.info(f"Final attempt for {f.__name__}.")
            return f(*args, **kwargs)
        return f_retry
    return deco_retry


class ContradictionEngine:
    """
    Hardened Contradiction Engine with ledger, tracer, and audit integration.
    """

    def __init__(
        self,
        *,
        ledger: ILedger,
        ruleset: Optional[List[Callable[[Dict[str, Any]], Optional[Dict[str, Any]]]]] = None,
        signing_key_hex: str,
        nonce_registry: NonceRegistry,
        revocation_registry: RevocationRegistry,
        name: str = "contradiction_engine",
        verify_strict: bool = True,
        quarantine_path: str = "data/quarantine.jsonl",
        metabolize_fn: Optional[Callable[[Dict[str, Any]], Optional[Dict[str, Any]]]] = None,
    ):
        logger.info(f"Initializing ContradictionEngine '{name}'.")
        if not all([ledger, signing_key_hex, nonce_registry, revocation_registry]):
            logger.error("Missing required initialization parameters.")
            raise ContradictionEngineError(
                "Ledger, signing_key, NonceRegistry, and RevocationRegistry are required."
            )

        self.name = name
        self.ruleset = ruleset or []
        self.ledger = ledger
        self.nonce_registry = nonce_registry
        self.revocation_registry = revocation_registry
        self.verify_strict = verify_strict
        self.metabolize_fn = metabolize_fn
        self._lock = threading.Lock()

        try:
            # Using bytes.fromhex to convert hex string to bytes
            self.signing_key = SigningKey(bytes.fromhex(signing_key_hex))
            self.verify_key = self.signing_key.verify_key.encode(encoder=HexEncoder).decode()
            logger.info("Successfully initialized signing and verification keys.")
        except Exception as e:
            logger.critical(f"Failed to initialize signing keys: {e}", exc_info=True)
            raise ContradictionEngineError("Failed to initialize signing keys.") from e


        sandbox_root = Path("data/sandbox")
        try:
            # Ensure the sandbox root exists
            sandbox_root.mkdir(parents=True, exist_ok=True)
            self._quarantine_path = ensure_in_sandbox(Path(quarantine_path), sandbox_root)
            logger.info(f"Quarantine path set to: {self._quarantine_path}")
        except Exception as e:
             logger.critical(f"Failed to set up sandboxed quarantine path: {e}", exc_info=True)
             raise ContradictionEngineError("Failed to set up sandboxed quarantine path.") from e


        # Initialize runtime tracer (asynchronous, non-blocking)
        # Wrap tracer initialization in try-except as it's not critical path
        self.tracer = None # Initialize to None
        try:
            self.tracer = Tracer(
                ledger=self.ledger,
                private_key_hex=signing_key_hex,
                executor_id=self.name
            )
            logger.info("Tracer initialized successfully.")
        except Exception as e:
             logger.error(f"Failed to initialize tracer: {e}", exc_info=True)
             # Engine can continue without tracer, so don't raise


        self._running = True
        self._monitor = ResourceMonitor("ContradictionEngine")
        logger.info(f"ContradictionEngine '{self.name}' initialization complete.")


    # --------------------------------------------------------
    # Core Methods
    # --------------------------------------------------------

    def _sign_payload(self, payload: Dict[str, Any]) -> Dict[str, Any]:
        """Sign a payload using the engine's key."""
        try:
            serialized = json.dumps(payload, sort_keys=True).encode()
            # Use .sign without .signature to get the signed message object
            signed_message = self.signing_key.sign(serialized)
            payload["signature"] = signed_message.signature.hex() # Extract hex signature
            payload["verify_key"] = self.verify_key
            logger.debug("Payload signed successfully.")
            return payload
        except Exception as e:
            logger.error(f"Failed to sign payload: {e}", exc_info=True)
            raise ContradictionEngineError("Failed to sign payload.") from e


    @Tracer.trace
    def _verify_event(self, event: Dict[str, Any]) -> bool:
        """Verifies receipt validity."""
        receipt = event.get("receipt")
        event_id = event.get('id', 'N/A')
        logger.debug(f"Verifying receipt for event: {event_id}")
        if not receipt:
            logger.warning(f"Event {event_id} missing receipt.")
            # Depending on your policy, missing receipt might be a VerificationError
            # raise VerificationError(f"Event {event_id} missing receipt.")
            return False # Allow processing events without receipts, but mark as unverified


        try:
            is_valid = verify_receipt(receipt, strict=self.verify_strict)
            if not is_valid:
                logger.warning(f"Receipt verification failed for event: {event_id}")
            else:
                 logger.debug(f"Receipt verification succeeded for event: {event_id}")
            return is_valid
        except Exception as e: # Catch potential errors within verify_receipt
            logger.error(f"Error during receipt verification for event {event_id}: {e}", exc_info=True)
            # Raising VerificationError indicates a problem with the verification process itself
            raise VerificationError(f"Receipt verification failed for event {event_id}: {e}") from e


    @Tracer.trace
    def _detect(self, event: Dict[str, Any]) -> List[Dict[str, Any]]:
        """Runs all rules against event payload to detect contradictions."""
        contradictions = []
        event_id = event.get('id', 'N/A')
        logger.debug(f"Running contradiction detection rules for event: {event_id}")
        for rule in self.ruleset:
            rule_name = rule.__name__ if hasattr(rule, '__name__') else 'anonymous_rule'
            try:
                logger.debug(f"Executing rule '{rule_name}' for event: {event_id}")
                result = rule(event)
                if result:
                    logger.info(f"Rule '{rule_name}' detected contradiction for event {event_id}.")
                    contradictions.append(result)
            except Exception as e:
                logger.error(f"Rule '{rule_name}' execution failed for event {event_id}: {e}", exc_info=True)
                # Log the error and continue to the next rule.
                # If a single rule failure should stop processing, raise RuleExecutionError here.
        logger.debug(f"Finished running contradiction detection rules for event: {event_id}. Found {len(contradictions)}.")
        return contradictions


    @Tracer.trace
    def _quarantine(self, event: Dict[str, Any], reason: str) -> None:
        """Write quarantined event to durable forensic log."""
        event_id = event.get('id', 'N/A')
        logger.warning(f"Quarantining event {event_id} due to: {reason}")
        try:
            record = {
                "timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
                "reason": reason,
                "event": event,
            }
            # Ensure directory exists before writing
            self._quarantine_path.parent.mkdir(parents=True, exist_ok=True)
            with open(self._quarantine_path, "a", encoding="utf-8") as f:
                json.dump(record, f) # Use json.dump for robustness
                f.write("\n")
            logger.info(f"Event {event_id} quarantined successfully to {self._quarantine_path}")
        except Exception as e:
            logger.critical(f"Failed to quarantine event {event_id}: {e}", exc_info=True)
            # Raising QuarantineViolation here is appropriate as it's a critical failure
            raise QuarantineViolation(f"Failed to quarantine event {event_id}: {e}") from e

    @Tracer.trace
    @retry(exceptions=LedgerInteractionError, tries=5, delay=2, backoff=3) # Apply retry to ledger interaction
    def _emit(self, contradiction: Dict[str, Any]) -> None:
        """Sign and emit contradiction; ledger maintains its own chain."""
        contradiction_id = contradiction.get('id', 'N/A') # Use contradiction id if available
        logger.info(f"Attempting to emit contradiction {contradiction_id} to ledger.")
        base = {
            "timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
            "type": "contradiction",
            "payload": contradiction,
        }
        try:
            signed = self._sign_payload(base)
        except ContradictionEngineError as e:
            logger.error(f"Failed to sign contradiction {contradiction_id} before emitting: {e}", exc_info=True)
            # Re-raise signing failure as it's a critical step before ledger interaction
            raise e

        try:
            # Check if ledger and add_event method exist and are callable
            if not hasattr(self.ledger, 'add_event') or not callable(self.ledger.add_event):
                logger.error("Ledger object missing or has uncallable `add_event()` method.")
                raise LedgerInteractionError("Ledger missing or has uncallable `add_event()` method.")
            self.ledger.add_event(signed)
            logger.info(f"Contradiction {contradiction_id} successfully added to ledger.")
        except LedgerInteractionError:
            # Re-raise LedgerInteractionError so the retry decorator can catch it
            raise
        except Exception as e:
            logger.error(f"Ledger interaction failed for contradiction {contradiction_id}: {e}", exc_info=True)
            # Wrap other ledger errors in our custom exception for retry decorator
            raise LedgerInteractionError(f"Failed to add contradiction {contradiction_id} to ledger: {e}") from e

        if self.metabolize_fn:
            try:
                logger.debug(f"Executing metabolism function for contradiction {contradiction_id}.")
                self.metabolize_fn(contradiction)
                logger.debug(f"Metabolism function executed for contradiction {contradiction_id}.")
            except Exception as e:
                logger.warning(f"Metabolism function failed for contradiction {contradiction_id}: {e}", exc_info=True)
                # Metabolism failure is logged but does not stop the process

        # CONTRADICTION_EVENTS_PROCESSED.inc() # Increment counter here after successful emission
        # Check if CONTRADICTION_EVENTS_PROCESSED is a Counter metric before incrementing
        if isinstance(CONTRADICTION_EVENTS_PROCESSED, Counter):
             CONTRADICTION_EVENTS_PROCESSED.inc()
             logger.debug("Contradiction processed count incremented.")
        else:
             logger.warning("Metrics counter 'CONTRADICTION_EVENTS_PROCESSED' is not a valid Counter instance.")


    # --------------------------------------------------------
    # Batch / Loop
    # --------------------------------------------------------

    def _run_once_unlocked(self, event: Dict[str, Any]) -> None:
        """Runs the full pipeline for a single event."""
        event_id = event.get('id', 'N/A')
        logger.info(f"Processing event: {event_id}")
        try:
            # _verify_event can raise VerificationError or return False
            if not self._verify_event(event):
                # If verification returns False, quarantine the event with a specific reason
                self._quarantine(event, "Receipt verification failed (returned False)")
                logger.warning(f"Event {event_id} quarantined due to verification failure.")
                return
        except VerificationError as e:
             logger.error(f"Critical verification error for event {event_id}: {e}", exc_info=True)
             self._quarantine(event, f"Critical verification error: {e}")
             return
        except Exception as e:
             # Catch any other unexpected errors during verification step
             logger.critical(f"Unexpected error during verification for event {event_id}: {e}", exc_info=True)
             self._quarantine(event, f"Unexpected verification error: {e}")
             return


        try:
            contradictions = self._detect(event)
            if contradictions:
                logger.info(f"Detected {len(contradictions)} contradictions for event {event_id}.")
            else:
                 logger.info(f"No contradictions detected for event {event_id}.")
        except Exception as e: # Catch any error during detection (including potential RuleExecutionError from rules)
             logger.error(f"Error during contradiction detection for event {event_id}: {e}", exc_info=True)
             # Decide if detection error should quarantine the event
             self._quarantine(event, f"Error during detection: {e}")
             return


        for c in contradictions:
            try:
                self._emit(c)
            except (LedgerInteractionError, ContradictionEngineError) as e:
                # These are errors we specifically handle (ledger failure after retries, or signing failure)
                logger.error(f"Failed to emit contradiction for event {event_id} after retries or signing error: {e}", exc_info=True)
                # Quarantining the original event that led to the contradiction
                self._quarantine(event, f"Failed to emit contradiction ({c.get('id', 'N/A')}): {e}")
            except Exception as e:
                 # Catch any other unexpected errors during emission
                 logger.critical(f"Unexpected error during emission for event {event_id}: {e}", exc_info=True)
                 # Quarantining the original event
                 self._quarantine(event, f"Unexpected emission error: {e}")


        logger.info(f"Finished processing event: {event_id}")


    def run_batch(self, events: List[Dict[str, Any]]) -> None:
        """Processes multiple events with quarantine fault tolerance."""
        logger.info(f"Processing batch of {len(events)} events.")
        with self._lock:
            for i, ev in enumerate(events):
                event_id = ev.get('id', f'batch_idx_{i}')
                try:
                    logger.debug(f"Processing event {event_id} in batch.")
                    self._run_once_unlocked(ev)
                except QuarantineViolation as e:
                    # This is a critical failure - quarantine itself failed
                    logger.critical(f"Quarantine write failure processing event {event_id}: {e}. Stopping engine.", exc_info=True)
                    print(f"[CRITICAL] Quarantine write failure: {e}")
                    self.stop()
                    break # Stop processing the batch if quarantine fails
                except Exception as e:
                     # Catch any unexpected errors during the batch processing loop for a single event
                     logger.critical(f"Unexpected error processing event {event_id} in batch: {e}", exc_info=True)
                     # Log and continue to the next event in the batch


    def run_forever(self, event_source: Callable[[], Dict[str, Any]], delay: float = 1.0):
        """Continuously polls event source and processes in real time."""
        logger.info(f"ContradictionEngine '{self.name}' running in forever mode.")
        print(f"[INFO] {self.name} running...")
        while self._running:
            try:
                event = event_source()
                if event:
                    # logger.debug("Received event from source.") # Too noisy if event source is polled frequently
                    self.run_batch([event])
                else:
                    # logger.debug("No event from source, sleeping.") # Too noisy
                    pass # Just sleep if no event
                time.sleep(delay)
            except KeyboardInterrupt:
                logger.info("Keyboard interrupt received. Stopping engine.")
                self.stop()
            except Exception as e:
                logger.error(f"Runtime loop exception: {e}", exc_info=True)
                # Decide if runtime loop exception should be quarantined or just logged
                # Currently logs and continues, but could add quarantine here if needed
                # self._quarantine({"error": str(e), "traceback": traceback.format_exc()}, "Runtime loop failure")

        logger.info(f"ContradictionEngine '{self.name}' has stopped.")


    def stop(self):
        """Stops the main loop and shuts down tracer thread."""
        logger.info(f"ContradictionEngine '{self.name}' is stopping.")
        print(f"[INFO] {self.name} stopping...")
        self._running = False
        try:
            # Give the tracer a moment to finish any pending tasks
            if self.tracer and hasattr(self.tracer, 'stop'):
                 self.tracer.stop()
                 logger.info("Tracer stop requested.")
            # self.tracer.join(timeout=5) # If tracer was a joinable thread, join it
        except Exception as e:
            logger.error(f"Failed to stop tracer cleanly: {e}", exc_info=True)
        logger.info(f"ContradictionEngine '{self.name}' stop process complete.")

    # --------------------------------------------------------
    # Verification / Stats
    # --------------------------------------------------------

    def verify_contradiction_chain(self) -> bool:
        """Delegates full verification to the ledger."""
        logger.info("Initiating ledger chain verification.")
        if not hasattr(self.ledger, "verify_chain") or not callable(self.ledger.verify_chain):
            logger.error("Ledger object missing or has uncallable `verify_chain()` method.")
            raise ContradictionEngineError("Ledger missing or has uncallable `verify_chain()` method.")
        try:
            is_valid = self.ledger.verify_chain()
            if is_valid:
                logger.info("Ledger chain verification succeeded.")
            else:
                logger.warning("Ledger chain verification failed.")
            return is_valid
        except Exception as e:
            logger.error(f"Error during ledger chain verification: {e}", exc_info=True)
            raise LedgerInteractionError(f"Ledger chain verification failed: {e}") from e


    def get_stats(self) -> Dict[str, Any]:
        """Reports engine health and metrics."""
        logger.info("Gathering engine stats.")
        stats = {
            "total_contradictions": 0,
            "total_scars": 0,
            "quarantine_size_bytes": 0,
            "chain_valid": False,
            "engine_running": self._running,
            "status": "ok"
        }
        try:
            if hasattr(self.ledger, 'get_all_events') and callable(self.ledger.get_all_events):
                try:
                    all_events = self.ledger.get_all_events(verify=False)
                    stats["total_contradictions"] = len([e for e in all_events if e.get("type") == "contradiction"])
                    stats["total_scars"] = len([e for e in all_events if e.get("type") == "scar"])
                except Exception as e:
                    logger.error(f"Failed to get events from ledger for stats: {e}", exc_info=True)
                    stats["status"] = "error_ledger_stats"

            if self._quarantine_path.exists():
                try:
                    stats["quarantine_size_bytes"] = os.path.getsize(self._quarantine_path)
                except Exception as e:
                    logger.warning(f"Failed to get quarantine file size: {e}")
                    stats["quarantine_size_bytes"] = -1 # Indicate error getting size
                    stats["status"] = "warning_quarantine_size"


            try:
                 stats["chain_valid"] = self.verify_contradiction_chain()
            except LedgerInteractionError as e:
                 logger.error(f"Failed to verify chain for stats: {e}", exc_info=True)
                 stats["chain_valid"] = False # Assume invalid if verification fails
                 stats["status"] = "error_chain_verify_stats"
            except Exception as e:
                 logger.error(f"Unexpected error verifying chain for stats: {e}", exc_info=True)
                 stats["chain_valid"] = False # Assume invalid if verification fails
                 stats["status"] = "error_chain_verify_stats_unexpected"


            logger.info(f"Engine stats gathered: {stats}")
            return stats
        except Exception as e:
            logger.error(f"Unexpected error during stats gathering: {e}", exc_info=True)
            stats["status"] = "error_unexpected_stats"
            # Decide if you raise the exception or return partial stats
            # raise ContradictionEngineError("Failed to gather engine stats.") from e
            return stats # Return partial stats in case of unexpected error


## Refine quarantine and metabolize functions

### Subtask:
Review and potentially improve the `_quarantine` and `_emit` methods, ensuring that quarantined events are handled securely and that the metabolism function is robust.


**Reasoning**:
Review and improve the `_quarantine` and `_emit` methods as per the instructions, focusing on secure handling of the quarantine file, robust writing, and graceful handling of the metabolism function's potential errors. Then, finish the task.

